# Langkah 1: Menyiapkan Data Pelatihan

**Asumsi seseorang dapat didiagnosa mengalami penyakit influenza (flu).**

Dari delapan rekam medis ini, kita dapat mengidentifikasi dua **kelas** utama yang ingin kita prediksi: _flu?='Y'_ (Ya, pasien ini memiliki flu) dan _flu?='N'_ (Tidak, pasien ini tidak memiliki flu).

Langkah pertama yang dilakukan Naive Bayes adalah membentuk "keyakinan awal" atau **prior probability** terhadap setiap kelas. Ini adalah frekuensi kemunculan setiap kelas dalam seluruh data pelatihan.

- Kita menemukan ada lima pasien yang didiagnosis _flu?='Y'_.
- Ada tiga pasien yang didiagnosis _flu?='N'_.
- Total pasien adalah delapan.

Dengan demikian, prior probability yang kita dapatkan sebagai berikut.

\begin{aligned}
P(\text{flu} = Y) &= \frac{\text{Jumlah pasien flu}}{\text{Total pasien}} = \frac{5}{8} = 0.625 \\
P(\text{flu} = N) &= \frac{\text{Jumlah pasien tidak flu}}{\text{Total pasien}} = \frac{3}{8} = 0.375
\end{aligned}


In [3]:
import pandas as pd
from collections import defaultdict
 
# 1. Data Pelatihan
print("1. Data Pelatihan")
data_train = {
   'chills': ['Y', 'Y', 'Y', 'Y', 'N', 'N', 'N', 'Y'],
   'runny nose': ['Y', 'Y', 'Y', 'Y', 'N', 'N', 'Y', 'Y'],
   'headache': ['Mild', 'No', 'Strong', 'Mild', 'No', 'Strong', 'Strong', 'Mild'],
   'fever': ['Y', 'Y', 'Y', 'Y', 'N', 'N', 'N', 'Y'],
   'flu?': ['Y', 'Y', 'Y', 'Y', 'N', 'N', 'N', 'Y']
   }

df_train = pd.DataFrame(data_train)
df_train

1. Data Pelatihan


,chills,runny nose,headache,fever,flu?
0,Y,Y,Mild,Y,Y
1,Y,Y,No,Y,Y
2,Y,Y,Strong,Y,Y
3,Y,Y,Mild,Y,Y
4,N,N,No,N,N
5,N,N,Strong,N,N
6,N,Y,Strong,N,N
7,Y,Y,Mild,Y,Y


# Langkah 2: Menghitung Probabilitas Prior

Setelah memiliki data latihnya, kita menghitung probabilitas prior dari flu?=Y dan flu?=N.

In [5]:
# Konfigurasi Naive Bayes
TARGET_VARIABLE = 'flu?'
FEATURES = ['chills', 'runny nose', 'headache', 'fever']
ALPHA = 1 # untuk Laplace Smoothing

# 2. Hitung Probabilitas Prior P(Kelas)
print("2. Probabilitas Prior P(Kelas)")
total_samples = len(df_train)
class_counts = df_train[TARGET_VARIABLE].value_counts()
classes = class_counts.index.tolist() # Mendapatkan nama kelas (Y, N)

priors = {}
for cls in classes:
    priors[cls] = class_counts[cls] / total_samples
    print(f"P({TARGET_VARIABLE}={cls}) = {priors[cls]:.4f}")

2. Probabilitas Prior P(Kelas)
P(flu?=Y) = 0.6250
P(flu?=N) = 0.3750


# Langkah 3: Menghitung Probabilitas Kondisional

Sederhananya, kita ingin mengetahui seberapa besar kemungkinan suatu gejala muncul pada pasien yang sudah diketahui kelasnya (flu atau tidak flu). Misalnya, "Seberapa sering pasien flu mengalami chills=Y?" atau "Seberapa sering pasien tanpa flu mengalami headache=Mild?"

Di dunia nyata, kadang ada kombinasi fitur dan kelas yang tidak muncul sama sekali dalam data pelatihan. Kalau langsung dihitung, hasilnya bisa nol, dan ini bisa merusak prediksi Naive Bayes. Untuk mengatasi ini, kita gunakan teknik Laplace smoothing, yaitu menambahkan angka kecil (biasanya 1) pada setiap kemungkinan supaya tidak ada probabilitas nol.

In [6]:
# --- 3. Hitung Probabilitas Kondisional (P(Fitur | Kelas)) dengan Laplace Smoothing ---
print("--- 3. Probabilitas Kondisional P(Fitur | Kelas) (dengan Laplace Smoothing) ---")
likelihoods = defaultdict(lambda: defaultdict(lambda: defaultdict(float)))
 
for feature in FEATURES:
   # Mendapatkan semua nilai unik untuk fitur ini dari data pelatihan
   unique_feature_values = df_train[feature].unique()
   num_unique_feature_values = len(unique_feature_values)
 
   for cls in classes:
       subset_df = df_train[df_train[TARGET_VARIABLE] == cls]
       total_in_class = len(subset_df)
 
       print(f"  Untuk Kelas '{cls}' (Jumlah sampel: {total_in_class}):")
       for value in unique_feature_values:
           # Hitung frekuensi fitur di dalam subset kelas
           count_feature_in_class = (subset_df[feature] == value).sum()
 
           # Terapkan Laplace Smoothing
           # Rumus: (count + alpha) / (total_in_class + alpha * num_unique_feature_values)
           likelihood = (count_feature_in_class + ALPHA) / \
                        (total_in_class + ALPHA * num_unique_feature_values)
 
           likelihoods[feature][value][cls] = likelihood
           print(f"    P({feature}={value} | {TARGET_VARIABLE}={cls}) = ({count_feature_in_class} + {ALPHA}) / ({total_in_class} + {ALPHA}*{num_unique_feature_values}) = {likelihood:.4f}")


--- 3. Probabilitas Kondisional P(Fitur | Kelas) (dengan Laplace Smoothing) ---
  Untuk Kelas 'Y' (Jumlah sampel: 5):
    P(chills=Y | flu?=Y) = (5 + 1) / (5 + 1*2) = 0.8571
    P(chills=N | flu?=Y) = (0 + 1) / (5 + 1*2) = 0.1429
  Untuk Kelas 'N' (Jumlah sampel: 3):
    P(chills=Y | flu?=N) = (0 + 1) / (3 + 1*2) = 0.2000
    P(chills=N | flu?=N) = (3 + 1) / (3 + 1*2) = 0.8000
  Untuk Kelas 'Y' (Jumlah sampel: 5):
    P(runny nose=Y | flu?=Y) = (5 + 1) / (5 + 1*2) = 0.8571
    P(runny nose=N | flu?=Y) = (0 + 1) / (5 + 1*2) = 0.1429
  Untuk Kelas 'N' (Jumlah sampel: 3):
    P(runny nose=Y | flu?=N) = (1 + 1) / (3 + 1*2) = 0.4000
    P(runny nose=N | flu?=N) = (2 + 1) / (3 + 1*2) = 0.6000
  Untuk Kelas 'Y' (Jumlah sampel: 5):
    P(headache=Mild | flu?=Y) = (3 + 1) / (5 + 1*3) = 0.5000
    P(headache=No | flu?=Y) = (1 + 1) / (5 + 1*3) = 0.2500
    P(headache=Strong | flu?=Y) = (1 + 1) / (5 + 1*3) = 0.2500
  Untuk Kelas 'N' (Jumlah sampel: 3):
    P(headache=Mild | flu?=N) = (0 + 1) / (3 

# Langkah 4: Prediksi — Pertarungan Probabilitas Posterior

Kini tibalah saatnya untuk mempraktikkan semua pengetahuan yang telah kita kumpulkan. Katakanlah seorang pasien baru masuk dengan gejala: chills='Y', runny nose='N', headache='Mild', dan fever='Y'. Pertanyaannya, apakah pasien ini memiliki flu?

In [7]:
# --- Data Pasien Baru untuk Prediksi ---
print("--- Data Pasien Baru ---")
# Gejala Pasien: chills=Y, runny nose=N, headache=Mild, fever=Y
new_patient_features = {
   'chills': 'Y',
   'runny nose': 'N',
   'headache': 'Mild',
   'fever': 'Y'
}
print(f"Gejala Pasien Baru: {new_patient_features}")
print("\n")
 
# --- 4. Prediksi untuk Pasien Baru ---
print("--- 4. Prediksi untuk Pasien Baru ---")
 
posterior_scores = {}
 
for cls in classes:
   # Mulai dengan probabilitas prior
   current_posterior = priors[cls]
   print(f"\n  Menghitung untuk Kelas '{cls}':")
   print(f"    P({TARGET_VARIABLE}={cls}) = {priors[cls]:.4f} (Prior)")
 
   # Kalikan dengan likelihoods dari setiap fitur pasien baru
   for feature in FEATURES:
       feature_value = new_patient_features[feature]
 
       # Ambil likelihood. Pastikan nilai fitur dari pasien baru ada dalam likelihoods yang dihitung.
       # Jika nilai fitur tidak ada dalam data pelatihan, ini akan menghasilkan KeyError tanpa defaultdict.
       # Dengan defaultdict, ini akan menghasilkan 0.0 yang kemudian + alpha.
       # Karena kita sudah menghitung semua nilai unik, ini harusnya tidak menjadi masalah.
       likelihood = likelihoods[feature][feature_value][cls]
 
       print(f"    P({feature}={feature_value} | {TARGET_VARIABLE}={cls}) = {likelihood:.4f}")
       current_posterior *= likelihood
 
   posterior_scores[cls] = current_posterior
   print(f"  Nilai Posterior (Proporsional) untuk '{cls}': {current_posterior:.8f}")

--- Data Pasien Baru ---
Gejala Pasien Baru: {'chills': 'Y', 'runny nose': 'N', 'headache': 'Mild', 'fever': 'Y'}


--- 4. Prediksi untuk Pasien Baru ---

  Menghitung untuk Kelas 'Y':
    P(flu?=Y) = 0.6250 (Prior)
    P(chills=Y | flu?=Y) = 0.8571
    P(runny nose=N | flu?=Y) = 0.1429
    P(headache=Mild | flu?=Y) = 0.5000
    P(fever=Y | flu?=Y) = 0.8571
  Nilai Posterior (Proporsional) untuk 'Y': 0.03279883

  Menghitung untuk Kelas 'N':
    P(flu?=N) = 0.3750 (Prior)
    P(chills=Y | flu?=N) = 0.2000
    P(runny nose=N | flu?=N) = 0.6000
    P(headache=Mild | flu?=N) = 0.1667
    P(fever=Y | flu?=N) = 0.2000
  Nilai Posterior (Proporsional) untuk 'N': 0.00150000


# Langkah Terakhir: Diagnosis Berbasis Keyakinan Terbaru

Dengan kedua skor probabilitas posterior di tangan, kita siap membuat keputusan akhir.

Skor untuk flu=Y: ≈ 0.0327

Skor untuk flu=N: ≈ 0.0015

In [8]:
# --- 5. Klasifikasi Akhir ---
print("\n--- 5. Klasifikasi Akhir ---")
predicted_class = max(posterior_scores, key=posterior_scores.get)
max_score = posterior_scores[predicted_class]
 
print(f"Skor untuk 'flu? = Y': {posterior_scores.get('Y', 0):.8f}")
print(f"Skor untuk 'flu? = N': {posterior_scores.get('N', 0):.8f}")
print(f"\nPasien baru ini kemungkinan besar memiliki flu: '{predicted_class}' (dengan skor {max_score:.8f})")


--- 5. Klasifikasi Akhir ---
Skor untuk 'flu? = Y': 0.03279883
Skor untuk 'flu? = N': 0.00150000

Pasien baru ini kemungkinan besar memiliki flu: 'Y' (dengan skor 0.03279883)
